### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import textwrap

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])



bins = [0, 69, 100]

group_names = ["Fail","Pass"]

school_data_complete["Pass/Fail (Math)"] = pd.cut(school_data_complete["math_score"], bins, labels=group_names)
school_data_complete["Pass/Fail (Reading)"] = pd.cut(school_data_complete["reading_score"], bins, labels=group_names)


#get the school_type
school_type = school_data[["school_name", "type"]]

#calculate the number of schools in the data set 
school_count = len(school_data_complete["school_name"].unique())

#calculate the number of students in the data set 
student_count = len(school_data_complete["Student ID"].unique())

# group data 
grouped_school_data = school_data_complete.groupby(["School ID", "school_name", "type"])

#calculate the overall sum of the budget from the data set 
school_budget = sum(grouped_school_data["budget"].mean())

#calculate the average math score
avg_math_score = school_data_complete["math_score"].mean()

#calculate the average reading score
avg_reading_score = school_data_complete["reading_score"].mean()

# calculate passing rate (overall_avergae)
passing_rate = (avg_math_score + avg_reading_score) / 2



math_pass_fail = pd.DataFrame(pd.cut(school_data_complete["math_score"], bins, labels=group_names))
reading_pass_fail = pd.DataFrame(pd.cut(school_data_complete["reading_score"], bins, labels=group_names))
students_who_passed_math = math_pass_fail[math_pass_fail["math_score"]=="Pass"].count()
students_who_passed_reading = reading_pass_fail[reading_pass_fail["reading_score"]=="Pass"].count()

#math_passes = math_pass_fail["math_score"] 

pct_passing_math = (students_who_passed_math["math_score"] /student_count) * 100
pct_passing_reading = (students_who_passed_reading["reading_score"] /student_count)  * 100


# Overall averall

district_summary_df = pd.DataFrame({"Total Schools" : [school_count] ,
                                    "Total Students" : [student_count],
                                    "Total Budget" : [school_budget],
                                    "Average Math Score" : [avg_math_score],
                                    "Average Reading Score" : [avg_reading_score],
                                    "% Passing Math" : [pct_passing_math],
                                    "% Passing Reading" : [pct_passing_reading],
                                    "% Overall Passing Rate" : [passing_rate] })

school_data_complete.head()

district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:.2f}".format)
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)



## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170",$24649428.00,78.985371,81.87784,74.980853,85.805463,80.431606


In [3]:
school_mean_data = grouped_school_data["budget"].mean()

#school_mean_data = grouped_student_data["budget"].mean()
school_student_count = grouped_school_data["Student ID"].count()

school_avg_math_score_data = grouped_school_data["math_score"].mean()


school_avg_reading_score_data = grouped_school_data["reading_score"].mean()

combined_mean_student_count = pd.merge(school_mean_data, school_student_count, on="school_name", how="inner")

combined_avg_math_and_reading = pd.merge(school_avg_math_score_data, school_avg_reading_score_data, on="school_name", how="inner")

school_results = pd.merge(combined_mean_student_count, combined_avg_math_and_reading, on="school_name", how="inner")

school_results.rename(columns={"school_name": "",
                                "Student ID": "Total Students",
                                "budget": "Total School Budget",
                                "math_score": "Average Math Score",
                                "reading_score": "Average Reading Score"
                                })

total_passing_math_by_school = school_data_complete.loc[school_data_complete["Pass/Fail (Math)"]=="Pass",:]
math_passes_grouped_by_school = total_passing_math_by_school.groupby(["School ID", "school_name", "type"])
math_passes_by_school = pd.DataFrame((math_passes_grouped_by_school["Pass/Fail (Math)"].count() / grouped_school_data["Student ID"].count()) * 100)

total_passing_reading_by_school = school_data_complete.loc[school_data_complete["Pass/Fail (Reading)"]=="Pass",:]
reading_passes_grouped_by_school = total_passing_reading_by_school.groupby(["School ID", "school_name", "type"])
reading_passes_by_school = pd.DataFrame((reading_passes_grouped_by_school["Pass/Fail (Reading)"].count() / grouped_school_data["Student ID"].count()) * 100)
#print(reading_passes_by_school)

math_reading_results_by_school_raw = pd.merge(math_passes_by_school, reading_passes_by_school, on ="school_name", how="inner"  )
math_reading_results = math_reading_results_by_school_raw.rename(columns={"0_x": "% Passing Math", "0_y": "% Passing Reading"})
math_reading_results["Overall Pass Rate"] = (math_reading_results["% Passing Math"] + math_reading_results["% Passing Reading"]) / 2



## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

,% Passing Math,% Passing Reading,Overall Pass Rate
school_name,,,
Cabrera High School,94.133477,97.039828,95.586652
Thomas High School,93.272171,97.308869,95.290520
Pena High School,94.594595,95.945946,95.270270
Griffin High School,93.392371,97.138965,95.265668
Wilson High School,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [5]:
math_reading_results.sort_values(by="Overall Pass Rate", ascending = True).head(5)

,% Passing Math,% Passing Reading,Overall Pass Rate
school_name,,,
Rodriguez High School,66.366592,80.220055,73.293323
Figueroa High School,65.988471,80.739234,73.363852
Huang High School,65.683922,81.316421,73.500171
Johnson High School,66.057551,81.222432,73.639992
Ford High School,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
school_data_complete.replace({"9th": "09th"}, inplace=True)

school_grade_groupby_data = school_data_complete.groupby(["school_name", "type", "grade"])

avg_math_score_by_school_and_grade = pd.DataFrame(school_grade_groupby_data["math_score"].mean())

avg_math_score_by_school_and_grade.sort_values(by=["school_name", "grade"], ascending = True)

math_score
school_name           type     grade            
Bailey High School    District 09th    77.083676
                               10th    76.996772
                               11th    77.515588
                               12th    76.492218
Cabrera High School   Charter  09th    83.094697
                               10th    83.154506
                               11th    82.765560
                               12th    83.277487
Figueroa High School  District 09th    76.403037
                               10th    76.539974
                               11th    76.884344
                               12th    77.151369
Ford High School      District 09th    77.361345
                               10th    77.672316
                               11th    76.918058
                               12th    76.179963
Griffin High School   Charter  09th    82.044010
                               10th    84.229064
                               11th    83.842105
                               12th    83.356164
Hernandez High School District 09th    77.438495
                               10th    77.337408
                               11th    77.136029
                               12th    77.186567
Holden High School    Charter  09th    83.787402
                               10th    83.429825
                               11th    85.000000
                               12th    82.855422
Huang High School     District 09th    77.027251
                               10th    75.908735
                               11th    76.446602
                               12th    77.225641
Johnson High School   District 09th    77.187857
                               10th    76.691117
                               11th    77.491653
                               12th    76.863248
Pena High School      Charter  09th    83.625455
                               10th    83.372000
                               11th    84.328125
                               12th    84.121547
Rodriguez High School District 09th    76.859966
                               10th    76.612500
                               11th    76.395626
                               12th    77.690748
Shelton High School   Charter  09th    83.420755
                               10th    82.917411
                               11th    83.383495
                               12th    83.778976
Thomas High School    Charter  09th    83.590022
                               10th    83.087886
                               11th    83.498795
                               12th    83.497041
Wilson High School    Charter  09th    83.085578
                               10th    83.724422
                               11th    83.195326
                               12th    83.035794
Wright High School    Charter  09th    83.264706
                               10th    84.010288
                               11th    83.836782
                               12th    83.644986

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
school_data_complete.replace({"9th": "09th"}, inplace=True)

school_grade_groupby_data = school_data_complete.groupby(["school_name", "type", "grade"])

avg_reading_score_by_school_and_grade = pd.DataFrame(school_grade_groupby_data["reading_score"].mean())

avg_reading_score_by_school_and_grade.sort_values(by=["school_name", "grade"], ascending = True)

reading_score
school_name           type     grade               
Bailey High School    District 09th       81.303155
                               10th       80.907183
                               11th       80.945643
                               12th       80.912451
Cabrera High School   Charter  09th       83.676136
                               10th       84.253219
                               11th       83.788382
                               12th       84.287958
Figueroa High School  District 09th       81.198598
                               10th       81.408912
                               11th       80.640339
                               12th       81.384863
Ford High School      District 09th       80.632653
                               10th       81.262712
                               11th       80.403642
                               12th       80.662338
Griffin High School   Charter  09th       83.369193
                               10th       83.706897
                               11th       84.288089
                               12th       84.013699
Hernandez High School District 09th       80.866860
                               10th       80.660147
                               11th       81.396140
                               12th       80.857143
Holden High School    Charter  09th       83.677165
                               10th       83.324561
                               11th       83.815534
                               12th       84.698795
Huang High School     District 09th       81.290284
                               10th       81.512386
                               11th       81.417476
                               12th       80.305983
Johnson High School   District 09th       81.260714
                               10th       80.773431
                               11th       80.616027
                               12th       81.227564
Pena High School      Charter  09th       83.807273
                               10th       83.612000
                               11th       84.335938
                               12th       84.591160
Rodriguez High School District 09th       80.993127
                               10th       80.629808
                               11th       80.864811
                               12th       80.376426
Shelton High School   Charter  09th       84.122642
                               10th       83.441964
                               11th       84.373786
                               12th       82.781671
Thomas High School    Charter  09th       83.728850
                               10th       84.254157
                               11th       83.585542
                               12th       83.831361
Wilson High School    Charter  09th       83.939778
                               10th       84.021452
                               11th       83.764608
                               12th       84.317673
Wright High School    Charter  09th       83.833333
                               10th       83.812757
                               11th       84.156322
                               12th       84.073171

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [9]:
school_budget_and_results_raw = pd.DataFrame(school_results)
school_budget_and_results_raw["average_spending"] = school_budget_and_results_raw["budget"] / school_budget_and_results_raw["Student ID"]

school_budget_and_spending_raw = pd.DataFrame(pd.cut(school_budget_and_results_raw["average_spending"], spending_bins, labels=group_names))

school_spending_vs_results = pd.merge(math_reading_results, school_budget_and_spending_raw, on="school_name", how="inner")
school_spending_vs_results.sort_values(by="Overall Pass Rate", ascending = False).head(5)



,% Passing Math,% Passing Reading,Overall Pass Rate,average_spending
school_name,,,,
Cabrera High School,94.133477,97.039828,95.586652,<$585
Thomas High School,93.272171,97.308869,95.290520,$615-645
Pena High School,94.594595,95.945946,95.270270,$585-615
Griffin High School,93.392371,97.138965,95.265668,$615-645
Wilson High School,93.867718,96.539641,95.203679,<$585


## Scores by School Size

* Perform the same operations as above, based on school size.

In [10]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [11]:

school_size_raw = pd.DataFrame(school_data)
school_size_raw.set_index('school_name', inplace=True)
school_size_raw = pd.DataFrame(pd.cut(school_size_raw["size"], size_bins, labels=group_names))

school_size_vs_results = pd.merge(math_reading_results, school_size_raw, on="school_name")
school_size_vs_results.sort_values(by="Overall Pass Rate", ascending = False).head(5)

,% Passing Math,% Passing Reading,Overall Pass Rate,size
school_name,,,,
Cabrera High School,94.133477,97.039828,95.586652,Medium (1000-2000)
Thomas High School,93.272171,97.308869,95.290520,Medium (1000-2000)
Pena High School,94.594595,95.945946,95.270270,Small (<1000)
Griffin High School,93.392371,97.138965,95.265668,Medium (1000-2000)
Wilson High School,93.867718,96.539641,95.203679,Large (2000-5000)


## Scores by School Type

* Perform the same operations as above, based on school type.

In [15]:
# Get Average Math and Reading Scores grouped by School Type
school_type_groupby_data = school_data_complete.groupby(["type"])

avg_math_score_by_type = pd.DataFrame(school_type_groupby_data["math_score"].mean())
avg_reading_score_by_type = pd.DataFrame(school_type_groupby_data["reading_score"].mean())
avg_math_reading_score_by_type = pd.merge(avg_math_score_by_type, avg_reading_score_by_type, on="type")

print(avg_math_reading_score_by_type)
# Rename Columns to be cleaner / more report friendly
school_data_complete_rename = pd.DataFrame(school_data_complete.rename(columns={"Pass/Fail (Math)": "Math_Results", "Pass/Fail (Reading)": "Reading_Results"}))

#Get a count of the Number of Students grouped by district and Math
school_data_complete_rename_math_grouby_data = school_data_complete_rename.groupby(["type"])
total_math_students_by_type_raw = pd.DataFrame(school_data_complete_rename_math_grouby_data["Math_Results"].count())
total_math_students_by_type = total_math_students_by_type_raw.rename(columns={"Math_Results": "total_math_students"})


#Get a count of the Number of Students grouped by district and Reading
school_data_complete_rename_reading_grouby_data = school_data_complete_rename.groupby(["type"])
total_reading_students_by_type_raw = pd.DataFrame(school_data_complete_rename_reading_grouby_data["Reading_Results"].count())
total_reading_students_by_type = total_reading_students_by_type_raw.rename(columns={"Reading_Results": "total_reading_students"})


# Get All math passes
all_math_passes = pd.DataFrame(school_data_complete_rename[school_data_complete_rename.Math_Results == 'Pass'])
all_math_passes_filtered_columns = all_math_passes[["type", "size", "Math_Results"]]
type_math_pass_groupby_data = all_math_passes_filtered_columns.groupby(["type"])
type_math_pass_raw = pd.DataFrame(type_math_pass_groupby_data["Math_Results"].count())
type_math_passes = type_math_pass_raw.rename(columns={"Math_Results": "total_math_passes"})

# Get All reading passes
all_reading_passes = pd.DataFrame(school_data_complete_rename[school_data_complete_rename.Reading_Results == 'Pass'])
all_reading_passes_filtered_columns = all_reading_passes[["type", "size", "Reading_Results"]]
type_reading_pass_groupby_data = all_reading_passes_filtered_columns.groupby(["type"])
type_reading_pass_raw = pd.DataFrame(type_reading_pass_groupby_data["Reading_Results"].count())
type_reading_passes = type_reading_pass_raw.rename(columns={"Reading_Results": "total_reading_passes"})


# Perform Merges
student_totals_by_subject_and_type = pd.merge(total_math_students_by_type, total_reading_students_by_type, on="type")
student_passes_by_subject_and_type = pd.merge(type_math_passes, type_reading_passes, on="type")
student_totals_by_type = pd.merge(student_passes_by_subject_and_type, student_totals_by_subject_and_type, on="type")
type_results_for_math_and_reading_raw = pd.merge(avg_math_reading_score_by_type, student_totals_by_type, on="type" )

# Compute Percentage Passes for Math and Reading by Type
type_results_for_math_and_reading_raw["% Passing Math"] = (type_results_for_math_and_reading_raw["total_math_passes"] / \
                                                          type_results_for_math_and_reading_raw["total_math_students"]) * 100

type_results_for_math_and_reading_raw["% Passing Reading"] = (type_results_for_math_and_reading_raw["total_reading_passes"] / \
                                                          type_results_for_math_and_reading_raw["total_reading_students"]) * 100

type_results_for_math_and_reading_raw["% Overall Passing Rate"] = ((type_results_for_math_and_reading_raw["total_math_passes"] + \
                                                          type_results_for_math_and_reading_raw["total_reading_passes"]) / \
                                                         (type_results_for_math_and_reading_raw["total_math_students"] + \
                                                          type_results_for_math_and_reading_raw["total_reading_students"] ) * 100)


type_results_for_math_and_reading_filtered = type_results_for_math_and_reading_raw.filter(["type", "math_score", "reading_score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"])
type_results_for_math_and_reading = type_results_for_math_and_reading_filtered.rename(columns={"type": "School Type", "math_score": "Average Math Score", "reading_score": "Average Reading Score"})
type_results_for_math_and_reading

#print(type_math_pass_fail_raw)
#avg_reading_score_by_type.sort_values(by=["school_name", "grade"], ascending = True)

          math_score  reading_score
type                               
Charter    83.406183      83.902821
District   76.987026      80.962485


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818


In [13]:
school_type_raw = pd.DataFrame(school_data)
school_type.set_index('school_name', inplace=True)
school_type_vs_results = pd.merge(math_reading_results, school_type, on="school_name")
school_type_vs_results.sort_values(by=["type", "Overall Pass Rate"], ascending = False)

,% Passing Math,% Passing Reading,Overall Pass Rate,type
school_name,,,,
Bailey High School,66.680064,81.933280,74.306672,District
Hernandez High School,66.752967,80.862999,73.807983,District
Ford High School,68.309602,79.299014,73.804308,District
Johnson High School,66.057551,81.222432,73.639992,District
Huang High School,65.683922,81.316421,73.500171,District
Figueroa High School,65.988471,80.739234,73.363852,District
Rodriguez High School,66.366592,80.220055,73.293323,District
Cabrera High School,94.133477,97.039828,95.586652,Charter
Thomas High School,93.272171,97.308869,95.290520,Charter
